In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
# チャットボットノードの作成
def build_graph(model_name):
    # ソースコードを記述
    llm = ChatOpenAI(model_name=model_name)

    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    llm_with_tools = llm.bind_tools(tools)

    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    graph_builder = StateGraph(State)
    graph_builder.add_node("chatbot", chatbot)

    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)
    graph_builder.add_conditional_edges("chatbot", tools_condition,)
    graph_builder.add_edge("tools", "chatbot")
    graph_builder.set_entry_point("chatbot")

    memory = MemorySaver()
    return graph_builder.compile(checkpointer=memory)


# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(graph, user_input)

こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？
1たす2は？
1たす2は3です。何か他に知りたいことがありますか？
台湾観光について検索結果を教えて

[{"url": "https://www.tabikobo.com/tabi-pocket/asia/taiwan/article65214.html", "content": "Image 8: 台北101 Image 14 Image 15: 九份 Image 21: 十分 Image 22: 台湾・十分【観光＆アクセス】徹底ガイド「美しいランタンが空を舞う街」に行ってみよう！ Image 23: 国立故宮博物院 Image 24: 士林夜市 Image 26: 龍山寺 Image 27: 台北101 Image 28: 徹底解説【台北101】はこんなビル！施設情報、人気の「展望台」、アクセス方法まで、まとめ Image 29: 淡水 Image 30: 行天宮 Image 31: 忠烈祠 Image 32: 中華民国総統府 Image 33 Image 36: パイナップルケーキ Image 37: 豆花 Image 38: 台湾を代表する【スイーツ10選】定番から流行り物まで！東京で味わえるお店も Image 39: 絶対食べたい「台湾グルメ」おすすめ｜台北人気店・夜市で食べるB級グルメも Image 40: 鼎泰豐の小籠包 Image 41: アイスモンスターのかき氷 Image 43: 串門子茶館のお茶 Image 48: 彩虹眷村 Image 49: 華山1914文化創意園区 Image 50: 北投温泉 Image 52: 四四南村 Image 53: 猫空ロープウェイ Image 54: 台中おすすめ日帰り観光スポット6選♪ Image 55: 【高雄】台湾好き大注目！定番＋穴場の「観光」スポット19選！一緒に行ける台南情報も♪ Image 60 *   Image 61 *   Image 62 *   Image 63 *   Image 64 *   Image 65 Image 66 Image 68 Image 69 *   Image 70 *   Image 71 *   Image 72 *   Image 73"}, {"url": "https://www.t